**Modelo Sinapsia**

In [1]:
'''
Install:
- pandas
- numpy
- ssqueezepy
- mne
- scipy
- sklearn
- tensorflow
'''

'\nInstall:\n- pandas\n- numpy\n- ssqueezepy\n- mne\n- scipy\n- sklearn\n- tensorflow\n'

In [1]:
import pandas as pd
import glob
import ssqueezepy #para convertir la señal en imagen
from ssqueezepy import cwt
from ssqueezepy.visuals import plot, imshow
import os
import mne #eeg analysis library
import scipy.io
#import torch.nn as nn
#import torch
import numpy as np
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Ordenando el dataset (no correr)**

----------> Juntamos los archivos

In [ ]:
#subo el archivo con las ids y diagnósticos (y)
metadata = pd.read_csv('metadata_guineabissau.csv')
metadata.head(5)

In [ ]:
metadata = metadata.drop('Remarks', axis=1)
metadata = metadata.drop('recordedPeriod', axis=1)
metadata = metadata.drop('startTime', axis=1)
metadata = metadata.drop('Eyes.condition', axis=1)

metadata.head(5)

In [ ]:
path = r'/content/eeg_data'
all_files = glob.glob(path + "/*.csv")

all_files = sorted(all_files)


def add_id_y (all_files, metadata):
  index = 0

  modified_dataframes_list = []
  non_eeg_columns = []

  for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    dataframe = df.iloc[:, 1:15].copy()
    dataframe['subject_id'] = metadata['subject.id'][index]
    dataframe['y'] = metadata['Group'][index]
    index += 1
    modified_dataframes_list.append(dataframe)

  return modified_dataframes_list

'''def join_csv (modified_dataframes_list):
  dataframes_list = []

  for filename in modified_dataframes_list:
    dataframe = pd.read_csv(filename, index_col=None, header=0)
    dataframes_list.append(dataframe)

  all_dataframes = pd.concat(dataframes_list, axis=0, ignore_index=True)
  all_dataframes.to_csv('eeg_data.csv', index=False)

  return all_dataframes'''

In [ ]:
modified_dataframes = add_id_y(all_files, metadata)

dataframes_list = []

for dataframe in modified_dataframes:
  dataframes_list.append(dataframe)

all_dataframes = pd.concat(dataframes_list, axis=0, ignore_index=True)
all_dataframes.to_csv('eeg_data.csv', index=False)

In [ ]:
all_dataframes

**---------------Procesando el dataset para el modelo (formato mne)---------------**

**Colmna y**

Epilepsia == 1

Normal == 0

In [10]:
eeg_data = pd.read_csv('C:\\Users\\47575909\\Desktop\\eeg_data.csv')
eeg_data

,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8,subject_id,y
0,4426.153846,3994.871795,4408.205128,3847.692308,4690.256410,3895.897436,4702.051282,3914.871795,4049.743590,4297.948718,4186.153846,4132.820513,4106.666667,4076.410256,1,Epilepsy
1,4420.512821,3986.666667,4394.358974,3836.923077,4678.461538,3886.666667,4696.410256,3910.769231,4054.358974,4295.897436,4187.179487,4126.666667,4105.641026,4065.128205,1,Epilepsy
2,4413.846154,3986.153846,4386.666667,3831.794872,4654.871795,3881.025641,4690.769231,3908.205128,4066.666667,4301.025641,4188.205128,4123.076923,4103.076923,4063.589744,1,Epilepsy
3,4407.692308,3984.615385,4384.102564,3832.820513,4644.615385,3883.076923,4686.153846,3910.256410,4063.076923,4298.974359,4184.615385,4127.179487,4095.384615,4071.282051,1,Epilepsy
4,4407.179487,3978.974359,4382.564103,3832.307692,4647.692308,3878.974359,4685.641026,3903.076923,4057.948718,4295.384615,4182.564103,4128.205128,4090.769231,4064.102564,1,Epilepsy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3850363,4211.794872,3862.051282,4139.487179,3670.256410,4327.179487,3662.564103,4557.435897,3894.358974,4238.974359,4189.743590,4010.256410,4126.153846,3830.769231,3883.589744,97,Control
3850364,4209.743590,3861.538462,4138.974359,3667.179487,4323.589744,3662.051282,4556.410256,3893.846154,4232.820513,4188.205128,4005.641026,4125.128205,3829.230769,3878.974359,97,Control
3850365,4211.282051,3862.564103,4136.410256,3671.282051,4324.102564,3664.102564,4556.923077,3896.923077,4235.384615,4185.128205,4005.128205,4123.076923,3829.230769,3882.051282,97,Control
3850366,4217.435897,3866.666667,4140.000000,3676.923077,4326.153846,3668.205128,4558.974359,3901.025641,4234.871795,4184.615385,4010.256410,4122.564103,3829.230769,3890.769231,97,Control


In [11]:
#paso los valores categóricos a numéricos
mapping = {'Epilepsy': 1, 'Control': 0}
eeg_data['y'] = eeg_data['y'].map(mapping)
eeg_data

,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8,subject_id,y
0,4426.153846,3994.871795,4408.205128,3847.692308,4690.256410,3895.897436,4702.051282,3914.871795,4049.743590,4297.948718,4186.153846,4132.820513,4106.666667,4076.410256,1,1
1,4420.512821,3986.666667,4394.358974,3836.923077,4678.461538,3886.666667,4696.410256,3910.769231,4054.358974,4295.897436,4187.179487,4126.666667,4105.641026,4065.128205,1,1
2,4413.846154,3986.153846,4386.666667,3831.794872,4654.871795,3881.025641,4690.769231,3908.205128,4066.666667,4301.025641,4188.205128,4123.076923,4103.076923,4063.589744,1,1
3,4407.692308,3984.615385,4384.102564,3832.820513,4644.615385,3883.076923,4686.153846,3910.256410,4063.076923,4298.974359,4184.615385,4127.179487,4095.384615,4071.282051,1,1
4,4407.179487,3978.974359,4382.564103,3832.307692,4647.692308,3878.974359,4685.641026,3903.076923,4057.948718,4295.384615,4182.564103,4128.205128,4090.769231,4064.102564,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3850363,4211.794872,3862.051282,4139.487179,3670.256410,4327.179487,3662.564103,4557.435897,3894.358974,4238.974359,4189.743590,4010.256410,4126.153846,3830.769231,3883.589744,97,0
3850364,4209.743590,3861.538462,4138.974359,3667.179487,4323.589744,3662.051282,4556.410256,3893.846154,4232.820513,4188.205128,4005.641026,4125.128205,3829.230769,3878.974359,97,0
3850365,4211.282051,3862.564103,4136.410256,3671.282051,4324.102564,3664.102564,4556.923077,3896.923077,4235.384615,4185.128205,4005.128205,4123.076923,3829.230769,3882.051282,97,0
3850366,4217.435897,3866.666667,4140.000000,3676.923077,4326.153846,3668.205128,4558.974359,3901.025641,4234.871795,4184.615385,4010.256410,4122.564103,3829.230769,3890.769231,97,0


In [12]:
#función para convertir los datos a formato mne
def convertDF2MNE(sub):
    info = mne.create_info(list(sub.columns), ch_types=['eeg'] * len(sub.columns), sfreq=128)
    info.set_montage('standard_1020')
    data=mne.io.RawArray(sub.T, info)
    data.set_eeg_reference()
    #data.filter(l_freq=0.1,h_freq=45)
    epochs = mne.make_fixed_length_epochs(data, duration = 3, overlap = 2)
    return epochs.get_data()

In [13]:
#agrupo las líneas que tienen el mismo id, o sea que son del mismo eeg
groups = eeg_data.groupby('subject_id')

#hago una lista con cada id de cada subject (hay 97 eeg)
grp_keys = list(groups.groups.keys())
print(grp_keys)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97]


In [14]:
from tqdm import tqdm

In [ ]:
#listas para almacenar los datos
data_list = []

for i, grpno in tqdm(enumerate(grp_keys)):
    grp = groups.get_group(grpno).drop(['subject_id'], axis=1)
    label = int(grp['y'].unique())
    subject_id = grpno
    grp = grp.drop('y', axis=1)

    #convierto mi info a formato mne
    data = convertDF2MNE(grp)  #(trials, channels, length)

    for c, x in enumerate(data):  #loop trials
        Wx, scales = cwt(x, 'morlet')
        Wx = np.abs(Wx)
        path = os.path.join('C:\\Users\\47575909\\Desktop\\scaleogram', f'subvideo_{grpno}/')
        os.makedirs(path, exist_ok=True)
        path = os.path.join(path, f'trial_{c}.npy')  # Corrección en la construcción de la ruta
        np.save(path, Wx)

        data_entry = {
            'grpno': i,
            'label': label,
            'path': path
        }

        #guardo la info del grupo (en realidad sería el "id" del eeg), la label y la path del archivo literal del eeg que ya se procesó
        data_list.append(data_entry)

#creo un dataset de TensorFlow
dataset = tf.data.Dataset.from_generator(
    generator=lambda: (data_entry for data_entry in data_list),
    output_signature={
        'grpno': tf.TensorSpec(shape=(), dtype=tf.int32),
        'label': tf.TensorSpec(shape=(), dtype=tf.int32),
        'path': tf.TensorSpec(shape=(), dtype=tf.string)
    }
)

In [16]:
#función para serializar un ejemplo en un TFRecord
def serialize(grpno, label, path):
    feature = {
        'grpno': tf.train.Feature(int64_list=tf.train.Int64List(value=[grpno])),
        'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
        'path': tf.train.Feature(bytes_list=tf.train.BytesList(value=[path.encode('utf-8')])),
    }
    
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    
    return example_proto.SerializeToString()

In [17]:
TFdataset = './tfrecord_data/'
os.makedirs(TFdataset)

#función para guardar los datos de cada grupo
def save_group_to_tfrecord(grpno, dataset):
    group_file_dataset = os.path.join(TFdataset, f'{grpno}_data.tfrecord')
    
    with tf.io.TFRecordWriter(group_file_dataset) as writer:
        for data_entry in dataset:
            grpno = data_entry['grpno'].numpy()
            label = data_entry['label'].numpy()
            path = data_entry['path'].numpy().decode('utf-8')
            serialized = serialize(grpno, label, path)
            writer.write(serialized)
    
    print(f'Datos del grupo {grpno} se guardó en {group_file_dataset}')

In [18]:
for grpno in grp_keys:
    group_dataset = dataset.filter(lambda x: x['grpno'] == grpno)
    save_group_to_tfrecord(grpno, group_dataset)

Datos del grupo 2 se guardó en ./tfrecord_data/2_data.tfrecord
Datos del grupo 3 se guardó en ./tfrecord_data/3_data.tfrecord
Datos del grupo 4 se guardó en ./tfrecord_data/4_data.tfrecord
Datos del grupo 5 se guardó en ./tfrecord_data/5_data.tfrecord
Datos del grupo 6 se guardó en ./tfrecord_data/6_data.tfrecord
Datos del grupo 7 se guardó en ./tfrecord_data/7_data.tfrecord
Datos del grupo 8 se guardó en ./tfrecord_data/8_data.tfrecord
Datos del grupo 9 se guardó en ./tfrecord_data/9_data.tfrecord
Datos del grupo 10 se guardó en ./tfrecord_data/10_data.tfrecord
Datos del grupo 11 se guardó en ./tfrecord_data/11_data.tfrecord
Datos del grupo 12 se guardó en ./tfrecord_data/12_data.tfrecord
Datos del grupo 13 se guardó en ./tfrecord_data/13_data.tfrecord
Datos del grupo 14 se guardó en ./tfrecord_data/14_data.tfrecord
Datos del grupo 15 se guardó en ./tfrecord_data/15_data.tfrecord
Datos del grupo 16 se guardó en ./tfrecord_data/16_data.tfrecord
Datos del grupo 17 se guardó en ./tfrecor

In [3]:
#ACA SUBO LOS ARCHIVOS DE TFRECORD
TFrecord_directory = 'C:\\Users\\47575909\\Desktop\\tfrecord_data' 

#lista de archivos TFRecord en la carpeta
filenames = [os.path.join(TFrecord_directory, f) for f in os.listdir(TFrecord_directory) if f.endswith('.tfrecord')]

#Hago un dataset de TensorFlow a partir de los archivos TFRecord
dataset = tf.data.TFRecordDataset(filenames)

#función para parsear los datos de TFRecord
def parse_tfrecord_fn(example):
    feature_description = {
        'grpno': tf.io.FixedLenFeature([], tf.int64),
        'label': tf.io.FixedLenFeature([], tf.int64),
        'path': tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, feature_description)
    return example

parsed_dataset = dataset.map(parse_tfrecord_fn)

#batching para argar los datos en lotes
batch_size = 16  #puedo cambiar el tamaño (en la PC del colegio el máximo es 16, pero capáz que le tengo que poner menos)
batched_dataset = parsed_dataset.batch(batch_size)

In [5]:
#ACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
#PRUEBO CON MENOS DATOS
#ACA SUBO LOS ARCHIVOS DE TFRECORD
TFrecord_directory = 'C:\\Users\\47575909\\Desktop\\tfrecord_data_prueba' 

#lista de archivos TFRecord en la carpeta
filenames = [os.path.join(TFrecord_directory, f) for f in os.listdir(TFrecord_directory) if f.endswith('.tfrecord')]

#Hago un dataset de TensorFlow a partir de los archivos TFRecord
dataset = tf.data.TFRecordDataset(filenames)

#función para parsear los datos de TFRecord
def parse_tfrecord_fn(example):
    feature_description = {
        'grpno': tf.io.FixedLenFeature([], tf.int64),
        'label': tf.io.FixedLenFeature([], tf.int64),
        'path': tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, feature_description)
    return example

parsed_dataset = dataset.map(parse_tfrecord_fn)

#batching para argar los datos en lotes
batch_size = 16  #puedo cambiar el tamaño (en la PC del colegio el máximo es 16, pero capáz que le tengo que poner menos)
batched_dataset = parsed_dataset.batch(batch_size)

#### Modelo (CNN) 

In [6]:
#SEPARO EN X E Y
X = []
y = []

#itero los lotes
for batch_data in batched_dataset:
    paths = batch_data['path'].numpy()
    labels = batch_data['label'].numpy()
    
    #cargo los scaleograms desde los archivos .npy
    batch_X = []
    for path in paths:
        spectrogram = np.load(path.decode('utf-8'))
        batch_X.append(spectrogram)
    
    #agrego los scaleograms y labels a las listas X y y
    X.extend(batch_X)
    y.extend(labels)

#convierto X e y a formato .npy
X = np.array(X)
y = np.array(y)

print('Ya se separó todo!')
print('Salió todo bien, tranquina :))')

Ya se separó todo!
Salió todo bien, tranquina :))


In [8]:
#NORMALIZO X en rango [-1, 1] para que el cero quede en el centro
min_val = np.min(X, axis = (0, 1, 2, 3))
max_val = np.max(X, axis = (0, 1, 2, 3))

X = -1 + 2 * (X - min_val) / (max_val - min_val)

#X = (X - X.min()) / (X.max() - X.min()) [0, 1]

In [10]:
X.shape

(606, 14, 230, 384)

In [11]:
#LOS GUARDO EN MI COMPUTADORA
#np.save('X.npy', X) está guardada normalizada en rango [0, 1], habría que cambiar eso
np.save('X_1.npy', X) #esto es para probar
np.save('y_1.npy', y)

In [3]:
# Cargar los arreglos X e y desde los archivos .npy (PARA CUANDO EL CÓDIGO ANTERIOR YA ESTÁ CORRIDO Y NO QUIERO VOLVERLO A CORRER :d)
X = np.load('C:\\Users\\47575909\\Desktop\\X_1.npy')
y = np.load('C:\\Users\\47575909\\Desktop\\y_1.npy')

In [4]:
X

array([[[[4.04229904e-05, 4.11818110e-05, 4.13666094e-05, ...,
          2.99415144e-04, 3.05189140e-04, 3.07502632e-04],
         [6.29508504e-05, 6.40436992e-05, 6.41800725e-05, ...,
          4.59725357e-04, 4.69591614e-04, 4.73476452e-04],
         [9.37218720e-05, 9.51857655e-05, 9.51344773e-05, ...,
          6.72586262e-04, 6.88805361e-04, 6.95081428e-04],
         ...,
         [1.50854327e-03, 1.51513040e-03, 1.52170344e-03, ...,
          2.58212164e-03, 2.58047506e-03, 2.57880450e-03],
         [1.10527975e-02, 1.10528385e-02, 1.10528767e-02, ...,
          1.10616479e-02, 1.10616311e-02, 1.10616125e-02],
         [3.53772826e-02, 3.53772864e-02, 3.53772826e-02, ...,
          3.53772901e-02, 3.53772864e-02, 3.53772901e-02]],

        [[2.45359497e-05, 2.37221666e-05, 2.33976261e-05, ...,
          9.41525141e-05, 9.33941556e-05, 9.13798212e-05],
         [3.37091915e-05, 3.18748789e-05, 3.10804571e-05, ...,
          1.42543155e-04, 1.41801895e-04, 1.38857155e-04],
        

In [23]:
#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_testVal, y_train, y_testVal = train_test_split(X, y, test_size=0.3, random_state=42)

#Divido los datos de test de los de evaluation (50/50 del 30% anterior)
X_val, X_test, y_val, y_test = train_test_split(X_testVal, y_testVal, test_size=0.5, random_state=42)

cwt.shape = (14, 230, 384)

- Alto: 230 (número de filas en tus imágenes CWT).
- Ancho: 384 (número de columnas en tus imágenes CWT).
- Canales: 14 (número de canales EEG).

In [ ]:
#si es que me toma mal la imput shape seguramente es por la longitud de la grabación de cada eeg, 
# por lo que dice la info del dataset no todos los eeg tienen la exacta misma duración

In [24]:
#CREO LA CNN
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', input_shape = (14, 230, 384)), 
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(32, activation='relu'),

    tf.keras.layers.Dense(1, activation='sigmoid') #función sigmoide porque yo quiero que me devuelva un 1 de "si epilepsia" o un 0 de "no epilepsia"
])

#(LA IMPUT SHAPE LA SAQUÉ CON LO DE Wx, AGARRÉ UN EJEMPLO DE UNA PRESONA Y ME FIJÉ EL TAMAÑO PORQUE YO LE VOY A DAR INFO DE PACIENTES DE ESE FORMATO A LA CNN)

In [25]:
#COMPILO
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [26]:
#ENTRENO
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
test_loss, test_acc = model.evaluate(X_val, y_val)

#capaz que debería probar con cross validation después

Epoch 1/10
67/67 [==============================] - 63s 828ms/step - loss: 0.6290 - accuracy: 0.6831 - val_loss: 0.4048 - val_accuracy: 0.8578
Epoch 2/10
67/67 [==============================] - 45s 677ms/step - loss: 0.3190 - accuracy: 0.8845 - val_loss: 0.2242 - val_accuracy: 0.9168
Epoch 3/10
67/67 [==============================] - 45s 680ms/step - loss: 0.1918 - accuracy: 0.9249 - val_loss: 0.1849 - val_accuracy: 0.9409
Epoch 4/10
67/67 [==============================] - 45s 666ms/step - loss: 0.1769 - accuracy: 0.9362 - val_loss: 0.1439 - val_accuracy: 0.9365
Epoch 5/10
67/67 [==============================] - 45s 676ms/step - loss: 0.1465 - accuracy: 0.9366 - val_loss: 0.1425 - val_accuracy: 0.9497
Epoch 6/10
67/67 [==============================] - 44s 663ms/step - loss: 0.1164 - accuracy: 0.9512 - val_loss: 0.1469 - val_accuracy: 0.9562
Epoch 7/10
67/67 [==============================] - 45s 679ms/step - loss: 0.1089 - accuracy: 0.9606 - val_loss: 0.1147 - val_accuracy: 0.9694

In [27]:
#VEO LA PERFORMANCE
print("Accuracy : ", test_acc)
print("Loss : ", test_loss)

Accuracy :  0.9518599510192871
Loss :  0.13316556811332703


In [ ]:
#investigar PCA para tensores (TAMBIÉN INVESTGAR MÁS PARA TENSORES)
#https://youtu.be/FgakZw6K1QQ?si=GrRc8tCp2eEfNKaX

#TAMBIÉN CHONA ME RECOMIENDA UNA LIBRERÍA PaCMACP para mi caso que uso tensores de tensorflow

**Tengo que hacer y notas**
##### **--> pasar todo a una pc**
##### **--> separar en datos de train, test y validation**

`78` = train

`10` = validation

`9` = test

##### **--> ver el formato y como feedearselo a una red neuronal**

Opciones de modelos a investigar:

*   `Convolutional neural network (CNN)`
*   `Recurrent neural network (RNN)`

